In [ ]:
import pickle
from graph import Graph, Part
from typing import List, Callable, Set, Tuple
from sklearn.model_selection import train_test_split

with open('data/graphs.dat', 'rb') as file:
    all_graphs: List[Graph] = pickle.load(file)
    X_train, X_temp, y_train, y_temp = train_test_split(list(map(lambda g: g.get_parts(), all_graphs)), all_graphs, test_size=0.3, random_state=0)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0) 

In [ ]:
import matplotlib.pyplot as plt

sizes = list(map(lambda graph: len(graph.get_nodes()), all_graphs))
counts, edges, bars = plt.hist(sizes, bins=range(min(sizes),max(sizes)+1,1))
plt.style.use('dark_background')
plt.bar_label(bars)
plt.show()

In [ ]:
from node import Node

class Ordering:
    def __init__(self, y: List[Graph], degreeAggregate: Callable[[List[int]], int]):
        degrees = {}
        for graph in y:
            for (node, edges) in graph.get_edges().items():
                if node.get_part().get_part_id() not in degrees:
                    degrees[node.get_part().get_part_id()] = []
                degrees[node.get_part().get_part_id()].append(len(edges))
        self.keys = {part: degreeAggregate(degs) for (part, degs) in degrees.items()}

    def sort(self, x: Set[Part]) -> List[Part]:
        return sorted(x, key=lambda n: self.keys[n.get_part_id()], reverse=True)
    
    def get_compatible_graphs(self, graphs: List[Graph], train_rate=False):
        compatible_graphs = []
        for graph in graphs:
            compatible = True
            appendOrder = [0]
            edges = {node: edges.copy() for (node, edges) in graph.get_edges().items()}
            nodes: List[Node] = sorted(graph.get_nodes(), key=lambda n: self.keys[n.get_part().get_part_id()])
            for node in nodes[:-1]:
                if len(edges[node]) != 1:
                    if train_rate:
                        self.keys[node.get_part().get_part_id()] += train_rate
                    compatible = False
                    break
                parent = edges[node][0]
                appendOrder.append(len(nodes) - nodes.index(parent) - 1)
                edges[parent].remove(node)
            if compatible:
                part_seq = list(map(lambda node: int(node.get_part().get_part_id()), nodes))
                compatible_graphs.append((graph, part_seq[::-1], appendOrder[::-1]))
        if train_rate:
            print(f"Training on {len(graphs)} graphs: ", end="")
        print(f"{len(compatible_graphs) / len(graphs)} accuracy")
        return compatible_graphs

avgOrder = Ordering(y_train, lambda n: sum(n)/len(n))
avgOrder.get_compatible_graphs(y_train, train_rate=0.1)
avgOrder.get_compatible_graphs(y_train, train_rate=0.1)
print("Train Accuracy: ", end="")
avgOrder_compatibleGraphs = avgOrder.get_compatible_graphs(y_train)
print("Validation Accuracy: ", end="")
avgOrder.get_compatible_graphs(y_val)
pass

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from datetime import datetime

class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.hidden2tag = nn.Linear(hidden_size, output_size)

    def forward(self, sentence):
        lstm_out, _ = self.lstm(sentence.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

def integers_to_onehot(numbers, output_size):
    one_hot = torch.zeros(len(numbers), output_size)
    for idx, number in enumerate(numbers):
        one_hot[idx, number] = 1
    return one_hot

# Define hyperparameters
input_size = 1 # Value between 0 and 2270
hidden_size = 20
output_size = 20
model = LSTM(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example training loop
for epoch in range(1000):
    total_loss = 0
    for graph, seq, positions in avgOrder_compatibleGraphs:

        input = torch.Tensor(list(map(lambda i: i/2270, seq)))
        target = integers_to_onehot(positions, output_size)

        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target) # Output had a .view(1,-1) in the example - why?
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if epoch % 100 == 0:
        print(f'{datetime.now().strftime("%H:%M:%S")}: Epoch {epoch}, Loss: {total_loss/10}')


In [ ]:

def createGraph(unorderedParts: Set[Part], model: nn.Module):
    parts = avgOrder.sort(unorderedParts)
    input = torch.Tensor(list(map(lambda part: int(part.get_part_id())/2270, parts)))
    output_one_hot = model(input)
    output_positions = torch.argmax(output_one_hot, dim=1).tolist()
    g = Graph()
    for idx, pos in enumerate(output_positions):
        if(pos >= len(parts)):
            pos = 0
        g.add_undirected_edge(parts[idx], parts[pos])
    if sum(output_positions) > 0:
        print(f"Non-Zero Model output for: {unorderedParts}")
    return g


correct_counter = 0
for parts, graph in zip(X_train, y_train):
    prediction = createGraph(parts, model)
    if prediction == graph:
        correct_counter += 1
print(correct_counter / len(y_train))

In [ ]:
import math

class MultiCount:
    def __init__(self):
        self.dict = {}

    def add(self, x1, x2):
        if x1 > x2:
            self.add(x2, x1)
        else:
            if x1 not in self.dict:
                self.dict[x1] = {}
            if x2 not in self.dict[x1]:
                self.dict[x1][x2] = 0
            self.dict[x1][x2] += 1
    
    def get(self, x1, x2):
        if x1 > x2:
            return self.get(x2, x1)
        else:
            return self.dict.get(x1, {}).get(x2, 0)

class Dendogramm:
    def __init__(self, y: List[Graph]):
        self.part_edge_count = MultiCount()
        self.part_part_count = MultiCount() # Just counts how often parts are together inside of a graph
        self.family_edge_count = MultiCount()
        for graph in y:
            for node, edges in graph.get_edges().items():
                for edge in edges:
                    self.part_edge_count.add(node.get_part().get_part_id(), edge.get_part().get_part_id())
                    self.family_edge_count.add(node.get_part().get_family_id(), edge.get_part().get_family_id())
            for part1 in graph.get_parts():
                for part2 in graph.get_parts():
                    self.part_part_count.add(part1.get_part_id(), part2.get_part_id())
    
    def _get_edge(self, part1: Part, part2: Part) -> float:
        part_part_count = self.part_part_count.get(part1.get_part_id(), part2.get_part_id())
        if part_part_count == 0:
            return 0
        return self.part_edge_count.get(part1.get_part_id(), part2.get_part_id()) / part_part_count
        #return self.part_edge_count.get(part1.get_part_id(), part2.get_part_id()) - 0.0 * self.family_edge_count.get(part1.get_family_id(), part2.get_family_id())

    def createGraph(self, parts: List[Part]) -> Graph:
        g = Graph()
        g.add_undirected_edge(parts[0], parts[1])
        for i in range(2, len(parts)):
            best_edge = max([(p, parts[i]) for p in parts[:i]], key=lambda p: self._get_edge(p[0], p[1]))
            g.add_undirected_edge(best_edge[0], best_edge[1])
        return g


testD = Dendogramm(y_train)
correct_train = 0
for i in range(len(y_train)):
    if y_train[i] == testD.createGraph(avgOrder.sort(y_train[i].get_parts())):
        correct_train += 1
correct_val = 0
for i in range(len(y_val)):
    if y_val[i] == testD.createGraph(avgOrder.sort(y_val[i].get_parts())):
        correct_val += 1
print(f"train_err: {correct_train / len(y_train)}")
print(f"val_err: {correct_val / len(y_val)}")
